In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
from transformers import BertTokenizer, TFBertModel
from sklearn.model_selection import train_test_split

# Load data
df = pd.read_csv('/content/spam.csv', encoding='latin-1', usecols=[0, 1], names=['label', 'text'])
df = df.drop(0)  # Remove header row
df['label'] = df['label'].map({'ham': 0, 'spam': 1})

# Clean text
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\d+', '', text)       # Remove numbers
    return text.strip()

df['text'] = df['text'].apply(clean_text)

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    df['text'], df['label'], test_size=0.3, random_state=42
)

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def encode_texts(texts, max_len=128):
    return tokenizer(
        texts.tolist(),
        padding='max_length',
        truncation=True,
        max_length=max_len,
        return_tensors='tf'
    )

train_encodings = encode_texts(X_train)
test_encodings = encode_texts(X_test)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Bidirectional, LSTM, Dense, Dropout, Layer
from tensorflow.keras.models import Model
from transformers import TFBertModel

# Custom BERT layer to handle symbolic tensor conversion
class BertLayer(Layer):
    def __init__(self, model_name='bert-base-uncased', **kwargs):
        super().__init__(**kwargs)
        self.bert = TFBertModel.from_pretrained(model_name)

    def call(self, inputs):
        input_ids, attention_mask = inputs
        output = self.bert(input_ids, attention_mask=attention_mask)
        return output.last_hidden_state

# BERT Inputs
input_ids = Input(shape=(128,), dtype=tf.int32, name='input_ids')
attention_mask = Input(shape=(128,), dtype=tf.int32, name='attention_mask')

# Use our custom BERT layer
bert_layer = BertLayer()
bert_output = bert_layer([input_ids, attention_mask])

# BiLSTM Layer
bilstm = Bidirectional(LSTM(128, return_sequences=True))(bert_output)

# Self-Attention Mechanism
attention = tf.keras.layers.Attention()([bilstm, bilstm])
pooled = tf.keras.layers.GlobalAveragePooling1D()(attention)

# Classifier Head
dense = Dense(64, activation='relu')(pooled)
dropout = Dropout(0.5)(dense)
output = Dense(1, activation='sigmoid')(dropout)

# Create model
model = Model(inputs=[input_ids, attention_mask], outputs=output)

# Compile model
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_ids           │ (None, 128)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_mask      │ (None, 128)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bert_layer          │ (None, 128, 768)  │          0 │ input_ids[0][0],  │
│ (BertLayer)         │                   │            │ attention_mask[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ (None, 128, 256)  │    918,528 │ bert_layer[0][0]  │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention           │ (None, 128, 256)  │          0 │ bidirectional[0]… │
│ (Attention)         │                   │            │ bidirectional[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 256)       │          0 │ attention[0][0]   │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │     16,448 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 64)        │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 1)         │         65 │ dropout[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 935,041 (3.57 MB)

 Trainable params: 935,041 (3.57 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
# Convert labels to tensors
train_labels = tf.convert_to_tensor(y_train.values, dtype=tf.float32)
test_labels = tf.convert_to_tensor(y_test.values, dtype=tf.float32)

# Train
history = model.fit(
    x={'input_ids': train_encodings['input_ids'], 'attention_mask': train_encodings['attention_mask']},
    y=train_labels,
    validation_split=0.2,
    epochs=10,
    batch_size=32
)

# Evaluate
results = model.evaluate(
    {'input_ids': test_encodings['input_ids'], 'attention_mask': test_encodings['attention_mask']},
    test_labels
)
print(f'Test Accuracy: {results[1]*100:.2f}%')

Epoch 1/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 56s 414ms/step - accuracy: 0.6522 - loss: 0.6271 - val_accuracy: 0.8654 - val_loss: 0.4022
Epoch 2/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 77s 386ms/step - accuracy: 0.8666 - loss: 0.3898 - val_accuracy: 0.8654 - val_loss: 0.3107
Epoch 3/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 43s 405ms/step - accuracy: 0.8685 - loss: 0.3120 - val_accuracy: 0.8808 - val_loss: 0.2512
Epoch 4/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 40s 394ms/step - accuracy: 0.8998 - loss: 0.2420 - val_accuracy: 0.9179 - val_loss: 0.2040
Epoch 5/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 41s 398ms/step - accuracy: 0.9175 - loss: 0.2041 - val_accuracy: 0.9397 - val_loss: 0.1687
Epoch 6/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 39s 399ms/step - accuracy: 0.9478 - loss: 0.1620 - val_accuracy: 0.9462 - val_loss: 0.1456
Epoch 7/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 41s 396ms/step - accuracy: 0.9561 - loss: 0.1398 - val_accuracy: 0.9551 - val_loss: 0.1306
Epoch 8/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 41s 397ms/step - accuracy: 0.9649 - loss: 0.1164 - val_accu

In [8]:
# Save entire model (TF SavedModel format)
model.save('my_bert_model.h5')
print("Model saved in 'my_bert_model' directory")

# Save weights only (alternative method)
model.save_weights('bert_model_weights.weights.h5')
print("Weights saved as 'bert_model_weights.weights.h5'")

Model saved in 'my_bert_model' directory
Weights saved as 'bert_model_weights.weights.h5'
